In [1]:
#imports
import pandas as pd
import logging
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from plotly import graph_objs as go
import numpy as np
from sklearn.metrics import mean_absolute_error

In [2]:
#read csv data
# november data
data_nov = pd.read_csv("https://raw.githubusercontent.com/iulianastroia/csv_data/master/final_dataframe.csv")
# october data
data = pd.read_csv("https://raw.githubusercontent.com/iulianastroia/csv_data/master/oct_data.csv")
# dec data
data_dec = pd.read_csv("https://raw.githubusercontent.com/iulianastroia/csv_data/master/dec_pm25.csv")

In [5]:
# combine df from oct,nov and dec
data = data.append(data_nov,sort=True)
data = data.append(data_dec,sort=True)
print("appended data", data)

appended data        altitude  ch2o    co2         day   latitude  longitude    o3  pm1  \
0        593.03  6.54  303.4    30/09/19  45.645893  25.599471  36.4  4.0   
1        593.12  5.54  318.4    30/09/19  45.645893  25.599471  38.4  4.0   
2        593.12  5.54  312.4    30/09/19  45.645893  25.599471  40.4  4.0   
3        593.12  6.54  302.4    30/09/19  45.645893  25.599471  38.4  4.0   
4        593.21  5.54  298.4    30/09/19  45.645893  25.599471  39.4  4.0   
...         ...   ...    ...         ...        ...        ...   ...  ...   
44540    477.56   NaN    NaN  31/12/2019  45.645893  25.599471   NaN  NaN   
44541    477.65   NaN    NaN  31/12/2019  45.645893  25.599471   NaN  NaN   
44542    477.56   NaN    NaN  31/12/2019  45.645893  25.599471   NaN  NaN   
44543    476.87   NaN    NaN  31/12/2019  45.645893  25.599471   NaN  NaN   
44544    476.69   NaN    NaN  31/12/2019  45.645893  25.599471   NaN  NaN   

       pm10  pm25  pressure        readable time  temperature

In [6]:
# drop Nan columns and indexes
data.dropna(axis='columns', how='all', inplace=True)
data.dropna(axis='index', how='all', inplace=True)

In [7]:
# convert to date format
data['day'] = pd.to_datetime(data['day'], dayfirst=True)

# sort dates by day
data = data.sort_values(by=['day'])
print("sorted days", data.day)

sorted days 0       2019-09-30
32      2019-09-30
33      2019-09-30
34      2019-09-30
35      2019-09-30
           ...    
44069   2019-12-31
44070   2019-12-31
44071   2019-12-31
44051   2019-12-31
44544   2019-12-31
Name: day, Length: 307317, dtype: datetime64[ns]


In [8]:
group_by_df = pd.DataFrame(
    [name, group.mean().pm25] for name, group in data.groupby('day')
)
group_by_df.columns = ['day', 'pm25']

In [10]:
# plot groupby df
fig = go.Figure(data=go.Scatter(x=group_by_df['day'], y=group_by_df['pm25']))
fig.update_layout(
    title='Pm2.5 REAL mean values for October-December',
    xaxis_title="Day",
    yaxis_title="Pm2.5")
fig.show()

In [16]:
# group df by day
grp_date = data.groupby('day')

# calculate mean value  for every given day
data = pd.DataFrame(grp_date.mean())
print("MEAN pm25 values by day\n", data.pm25)


MEAN pm25 values by day
 day
2019-09-30     8.100000
2019-10-01     3.363700
2019-10-02    11.816852
2019-10-03    15.225470
2019-10-04     3.624913
                ...    
2019-12-27     8.782669
2019-12-28     7.543493
2019-12-29     6.784969
2019-12-30     9.616562
2019-12-31    27.283229
Name: pm25, Length: 93, dtype: float64


In [17]:
# start using prophet
logging.getLogger().setLevel(logging.ERROR)

# create df for prophet
df = data.reset_index()
#ds=date, y=pm2.5 values
df.columns = ['ds', 'y']
print(df)

           ds          y
0  2019-09-30   8.100000
1  2019-10-01   3.363700
2  2019-10-02  11.816852
3  2019-10-03  15.225470
4  2019-10-04   3.624913
..        ...        ...
88 2019-12-27   8.782669
89 2019-12-28   7.543493
90 2019-12-29   6.784969
91 2019-12-30   9.616562
92 2019-12-31  27.283229

[93 rows x 2 columns]


In [18]:
m = Prophet()
m.fit(df)

# make predictions for next month(30 days)->january
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
# print only values of interest
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

            ds       yhat  yhat_lower  yhat_upper
0   2019-09-30  10.461550  -12.752007   36.013124
1   2019-10-01  17.000902   -7.857386   41.831068
2   2019-10-02  14.498700  -11.940635   41.351081
3   2019-10-03  14.316018  -11.636592   38.850552
4   2019-10-04  10.944340  -14.159963   36.326024
..         ...        ...         ...         ...
118 2020-01-26  30.891494    6.170266   53.775343
119 2020-01-27  37.115282   12.737967   61.612111
120 2020-01-28  43.654634   19.142028   66.925631
121 2020-01-29  41.152432   16.488296   63.837310
122 2020-01-30  40.969750   14.699256   65.490823

[123 rows x 4 columns]


In [19]:
# plot predictions
fig = plot_plotly(m, forecast)
fig.update_layout(
    title='Pm2.5 forecast for October 2019-January 2020',
    xaxis_title="Day",
    yaxis_title="Pm2.5")
fig.show()

In [20]:
# check if there is seasonality+trend
fig = plot_components_plotly(m, forecast)
fig.update_layout(
    title='Pm2.5 characteristics-seasonality'
)
fig.show()

In [21]:
# define a function to make a df containing the prediction and the actual values
def make_comparison_dataframe(historical, forecast):
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))


cmp_df = make_comparison_dataframe(df, forecast)
print("COMPARED DF of predicted and real values", cmp_df)

# plot forecast with upper and lower bound
fig = go.Figure()

# predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat'],
    name='yhat(predicted value)'
))

# lower bound of predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat_lower'],
    name='yhat_lower'
))

# upper bound of predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat_upper'],
    name='yhat_upper'
))

# actual value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['y'],
    name='y(actual value)'
))
fig.update_layout(title='Comparison between predicted values and real ones', yaxis_title='Pm2.5', xaxis_title='Day',
                  showlegend=True)
fig.show()


COMPARED DF of predicted and real values                  yhat  yhat_lower  yhat_upper          y
ds                                                      
2019-09-30  10.461550  -12.752007   36.013124   8.100000
2019-10-01  17.000902   -7.857386   41.831068   3.363700
2019-10-02  14.498700  -11.940635   41.351081  11.816852
2019-10-03  14.316018  -11.636592   38.850552  15.225470
2019-10-04  10.944340  -14.159963   36.326024   3.624913
...               ...         ...         ...        ...
2020-01-26  30.891494    6.170266   53.775343        NaN
2020-01-27  37.115282   12.737967   61.612111        NaN
2020-01-28  43.654634   19.142028   66.925631        NaN
2020-01-29  41.152432   16.488296   63.837310        NaN
2020-01-30  40.969750   14.699256   65.490823        NaN

[123 rows x 4 columns]


In [24]:
#errors
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


# don't take january values into consideration at MAPE and MAE
cmp_df = cmp_df.dropna()
print("MAPE ", mean_absolute_percentage_error(cmp_df['y'], cmp_df['yhat']))
print("MAE", mean_absolute_error(cmp_df['y'], cmp_df['yhat']))

forecast_errors = [abs(cmp_df['y'][i] - cmp_df['yhat'][i]) for i in range(len(cmp_df))]
print('\nForecast Errors: ', forecast_errors)
print('\nMAX Forecast Error: %s' % max(forecast_errors))
print('\nMIN Forecast Error: %s' % min(forecast_errors))


MAPE  111.3659786805749
MAE 13.825488423450745

Forecast Errors:  [2.3615498759559426, 13.63720261848588, 2.6818475899323424, 0.9094522007222139, 7.319426977938747, 8.912321629272215, 3.6550534290335905, 5.973049047677236, 8.478239557258378, 2.2864691363979794, 4.166220710510448, 8.19145187831321, 2.7847000283692065, 7.1428583626153115, 1.273370601401428, 3.3697599747572475, 0.22697408332082958, 7.433346756571499, 13.260677843302535, 10.687795232129176, 15.74987607423185, 12.87729985907501, 6.700922186345547, 7.17792307088315, 5.311162603671981, 9.592839597626112, 13.360434608045885, 21.901349232139104, 15.027067156655264, 43.80380189222161, 4.87135269666717, 11.51705158743532, 3.346850556287732, 4.879392394579304, 1.4159509333282898, 9.952935784131228, 21.960179433434963, 18.44589658250807, 13.603049292768459, 6.336597802271886, 9.593885804947545, 3.635902333702248, 12.413634677045751, 12.121592784528218, 10.989232914526744, 0.7545541987477442, 3.7356732263069716, 9.664444757536781, 4